# Time Series Introduction


Observational or experimental data observed at points in time are called time series. Answering analytical questions about these data is called time series analysis. There are two major approaches to time series analysis: time domain and frequency domain.

Time domain analysis views a time series as linear combination of past values of a noise series and a deterministic component (Wold, 1938).

Frequency domain analysis views the time series as the linear combination of sine and cosine series at different frequencies (periods) (Cramer, 1942).

This session introduces concepts from both approaches with implementations in python.

In [ ]:
%matplotlib inline

# # Printing
import locale


# # Data
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm

# # Time series

from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse



# # Plotting

import matplotlib.pyplot as plt

# sns.set(style="darkgrid", color_codes=True)






# Time Series Formats

Time is a scientific and cultural concept. Different cultures have chosen different approaches to measuring the passage of time and recording the current time. Some cultural examples:

- Time zones vs. local times
- Day light savings
- Solar vs. lunar calendars

Cultural differences and different scientific uses have produced many different formats for dates and times. Some examples:

 - M-D-Y, 
 - D-M-Y, 
 - D/M/Y, 
 - Hr:Min AM/PM (12 hour), 
 - Hr+Min (24 hour)
 - Can measure time as counts (seconds) from a start point, e.g., unix time measured from Jan 1 1970, minus leap seconds  

To do data science with temporal data requires parsers to read the different formats. 


## Explicit time formating

In [ ]:
# Functions to convert datetimes to strings
time = datetime.now()
str(time)

In [ ]:
time

In [ ]:
# Get just the date
time.strftime('%Y-%m-%d')

In [ ]:
# Going back to datetime
stime = time.strftime('%Y-%m-%d')
datetime.strptime(stime,'%Y-%m-%d')

## Time parser function

In [ ]:
# There is a parser

parse(stime)

In [ ]:
# the parser will work with lots of text
parse('Nov 24, 2017, 10:30 am')

In [ ]:
time.year, time.month, time.day

In [ ]:
dt = timedelta(5)
datetime.now() + dt

## Elements of a time series

- Trend
- Seasonality or periodicity
- Cycles
- Randomness

## Example - Airline Passenger Data

Get the airline passenger data from the url below and read it into a data frame. Look at its shape.

https://datamarket.com/data/set/22u3/international-airline-passengers-monthly-totals-in-thousands-jan-49-dec-60#!ds=22u3&display=line

In [ ]:
path = 
file = 
airlines = pd.read_csv(path+file,  header = 0, names = ['Date', 'Passengers'])
airlines.shape

In [ ]:
airlines.head()


In [ ]:
# Null values?

airlines.isnull().sum()

In [ ]:
# Get index for null value
airlines[airlines['Passengers'].isnull()].index.tolist()

In [ ]:
# Remove it

airlines.drop(144, inplace = True)

In [ ]:
# Making column Date into a datetime index.

airlines['Date'] = pd.to_datetime(airlines['Date'], format = "%Y-%m")

airlines.index = airlines['Date']
airlines.drop('Date', axis = 1, inplace = True)
airlines.head()

In [ ]:
# Plot the passenger data

plt.figure(figsize = (15,10))


airlines.Passengers.plot()
plt.xlabel('Date')
plt.ylabel('No. of Passengers (thousands)')
plt.title("International Airline Passengers")

In [ ]:
# Decomposing the elements of the time series


from statsmodels.tsa.seasonal import seasonal_decompose

#s=sm.tsa.seasonal_decompose(airlines.Passengers)

decomposition = seasonal_decompose(airlines.Passengers, freq = 12)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid



fig = plt.figure(figsize = (14,11))
plt.subplots_adjust(hspace = .5)

ts = fig.add_subplot(4, 1, 1)
airlines.Passengers.plot()
ts.set_title("Airline Passenger Data")
ts_trend = fig.add_subplot(4,1,2)
trend.plot()
ts_trend.set_title('Airline Passenger Trend')
ts_seasonal = fig.add_subplot(4,1,3)
seasonal.plot()
ts_seasonal.set_title('Airline Passenger Seasonality')
ts_residual = fig.add_subplot(4,1,4)
residual.plot()
ts_residual.set_title('Airline Passenger Residuals')

## Time domain filters

Remove some data elements in order to focus on other elements

- Low pass filters remove the high fequency components
- High pass filters remove the low fequency components

In [ ]:
# Filters - Moving Averages

# Plot moving (rolling) averages at 3, 6 and 12 months.
# Also plot the standard deviation for 3 months


fig = plt.figure(figsize = (14,11))
ts = fig.add_subplot(1, 1, 1)
airlines.Passengers.plot(label = 'Airline Passengers')
airlines.Passengers.rolling(window =  3, center = True).mean().plot(label = 'Rolling 3 Month Mean')
airlines.Passengers.rolling(window =  6, center = True).mean().plot(label = 'Rolling 6 Month Mean')
airlines.Passengers.rolling(window =  12, center = True).mean().plot(label = 'Rolling 12 Month Mean')
airlines.Passengers.rolling(window =  12, center = True).std().plot(label = 'Rolling 3 Month STD')

ts.legend(loc = 'best')
ts.set_title("International Airline Passengers")
ts.set_ylabel("No. of Passengers (Thousands)")
ts.set_xlabel("Years")


In [ ]:
# Filters - Differences

fig = plt.figure(figsize = (14,11))
ts_diff = fig.add_subplot(1,1,1)

airlines['Passengers'].diff(1).plot(label = 'First Difference')
airlines['Passengers'].diff(3).plot(label = '3rd Difference')
airlines['Passengers'].diff(6).plot(label = '6th Difference')
airlines['Passengers'].diff(12).plot(label = '12th Difference')

ts_diff.legend(loc = 'best')



## Stationarity


Given a time series $x_t, t = 1,2, \ldots, N$ we want to know and describe the relationship between $x_t$ and $x_s$ for $t,s \in \{1,2,\ldots,N$. We call $k$ for $x_{t-k}$ a lag of $k$ at time $t$. Similarly, we call $k$ for $x_{t+k}$ a lead of $k$ at time $t$.

The mean of the time series, $\mu$, is constant and the autocovariance, C, is a function only of the lag (or lead). So

$E[(x_t- \mu)(x_{t-k} - \mu)] = C(t,t-k) = C(k)$

In [ ]:
# Dickey-Fuller test of stationarity
# 

from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    #Perform Dickey-Fuller test:
    # Null hypothesis is that the data are non-stationary
    print ('Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput) 

In [ ]:
# Test airlines data for stationarity

test_stationarity(airlines['Passengers'])

## Autocorrelation and Partial Autocorrelation

For lag $k$ let $C(k)$ be the autocorrelation between $x_t, x_{t-k}, t = 1,2,\ldots,N$. Then the autocorrelation at $k$  is $\rho(k) = \frac{C(k)}{C(0)}$.

Let $z_t = x_t - E(x)$. The partial autocorrelation at lag $k$ is the last coefficient in the regression:
\begin{equation*}
z_t = \phi_{t-1}z_{t-1} + \phi_{t-2}z_{t-2} + \cdots \ + \phi_{t-k}z_{t-k} + \epsilon
\end{equation*}

In [ ]:

# Autocorrelation and partial autocorrelations for the airline data.

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(airlines['Passengers'], lags=20, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(airlines['Passengers'], lags=20, ax=ax2)

# Frequency Domain

Model the time series as Fourier frequencies with sine and cosine functions. Generally,

$x_t = \sum_{k=1}^Q [a_{1k}sin(2\pi\nu_kt) + a_{2k}cos(2\pi\nu_kt)]$

where $\nu$ is the frequency and the period is $1/\nu$. For computational reasons we use the
discrete Fourier transform:

$X(\nu_k) = n^{-\frac{1}{2}} \sum_{t=1}^n x_t exp(-2\pi i\nu_k t)  $

Graphs of the FFT give the periordogram which plots power vs. frequency

In [ ]:
# Get the FFT for the passenger series

passengers_fft = sp.fftpack.fft(airlines['Passengers'])
passengers_psd = np.abs(passengers_fft)**2

In [ ]:
# Filter by positive frequencies

passengers_fftfreq = sp.fftpack.fftfreq(len(passengers_psd), 1/12)
i = passengers_fftfreq > 0 

In [ ]:
# Plot the Periodogram and find the frequencies

plt.figure(figsize = (14,11))
plt.plot(passengers_fftfreq[i], 10*np.log10(passengers_psd[i]))
#plt.xlim(0,5)
plt.xlabel('Frequency (1/year)')
plt.ylabel('PSD (dB)')
plt.title('Spectral Density of Airline Passengers')

In [ ]:
# Remove all frequencies that are harmonics
# Invert the FFT to get a model of the time series
# Use only the real part

passengers_fft_bis = passengers_fft.copy() # get a copy
passengers_fft_bis[np.abs(passengers_fftfreq > 1.1)] = 0 # remove harmonics
passengers_freqfit = np.real(sp.fftpack.ifft(passengers_fft_bis)) #invert fft
passengers_freqfit = pd.Series(passengers_freqfit, index = airlines.index)

In [ ]:
# 
# Then plot on the original series
# with the frequency model

fig = plt.figure(figsize = (14,11))
ts = fig.add_subplot(1, 1, 1)
airlines['Passengers'].plot(label = 'Airline Passengers')
passengers_freqfit.plot(label = 'Frequency Filter')

ts.legend(loc = 'best')
ts.set_title("International Airline Passengers")
plt.xlabel('Dates')
plt.ylabel('No. of Passengers (thousands)')

# Exercises

Load each of the data sets (from Schumway, 2014):

    - speech.csv (speech recording)
    - liveBirths.txt (U.S. Monthly Live Births)
    - oil.csv (Crude oil, WTI spot price FOB)
    - globtemp.csv (Global mean land-ocean temperature deviations)
    - flu.csv (Monthly pneumonia and influenza deaths in the U.S., 1968 to 1978)
    
For each of these time series do the following

    1. Decompose to show the elements of the time series
    2. Filter based on moving averages (at least 3) and differences (at least the first)
    3. Plot the autocorrelations and partial autocorrelations
    4. Plot the periodogram and the frequencies with the original series
    5. Based on these filters and visualizations, comment on the characteristics (elements) of the time series
